This example is for buiding intuition concept of DRAG pulse [1], which aims to mitigate leakage to state 2 when we realize 0-1 transition in transmon. The relation between system system parameter and quantum dynamics can be seen from tunable figures based on suraida
[1] Motzoi, F. et al., Phys. Rev. Lett., 103, 110501 (2009)

In [1]:
from qutip import basis, destroy, sesolve, qeye, tensor, expect, QobjEvo
import qutip as qt
import numpy as np
import jax.scipy.special as jsp
import suraida as sr
import matplotlib.pyplot as plt
def DRAG(tlist, Kerr, tg ):
    dim = 3
    a = destroy(dim)
    Kerr = Kerr *2*np.pi
    H0 = 0.5 * Kerr * a.dag() * a.dag() * a * a
    H1s = [a + a.dag(), 1j * (a - a.dag())]

    time = tg
    tsave = tlist
    initial_states = [basis(dim, 0)]
    exp_ops = [basis(dim, idx) * basis(dim, idx).dag() for idx in range(dim)]

    def gaussian_env(t, sigma, matrix_element, tg, rot_angle=np.pi, ti=0):
        tf = ti + tg
        amp = rot_angle / (np.sqrt(2 * np.pi) * sigma * jsp.erf(tg / (2 * np.sqrt(2) * sigma)) - np.exp(-(tg / 2) ** 2 / (2 * sigma ** 2)) * tg) / matrix_element
        offset = amp * np.exp(-(tg / 2) ** 2 / (2 * sigma ** 2))
        return (amp * np.exp(-(t - ti - tg / 2) ** 2 / (2 * sigma ** 2)) - offset) * np.heaviside(t - ti, 0) * np.heaviside(tf - t, 0)

    def dev_gaussian_env(t, sigma, matrix_element, tg, rot_angle=np.pi, ti=0):
        tf = ti + tg
        amp = rot_angle / (np.sqrt(2 * np.pi) * sigma * jsp.erf(tg / (2 * np.sqrt(2) * sigma)) - np.exp(-(tg / 2) ** 2 / (2 * sigma ** 2)) * tg) / matrix_element
        d_amp = -amp * (t - ti - tg / 2) / (sigma ** 2) * np.exp(-(t - ti - tg / 2) ** 2 / (2 * sigma ** 2))
        return d_amp * np.heaviside(t - ti, 0) * np.heaviside(tf - t, 0)

    def I(t):
        tg = time
        sigma = tg / 3
        matrix_element = 1
        rot_angle = np.pi
        return gaussian_env(t, sigma, matrix_element, tg, rot_angle) / 2

    def Q(t):
        tg = time
        sigma = tg / 3
        matrix_element = 1
        rot_angle = np.pi
        return dev_gaussian_env(t, sigma, matrix_element, tg, rot_angle) / 2 / (2 * Kerr)

    def H_func(t, args):
        H = H0
        H += I(t) * H1s[0]
        H += Q(t) * H1s[1]
        return H

    H = QobjEvo(H_func, args={})
    options = {"store_states": True, "atol": 1e-12}
    result = sesolve(H, initial_states[0], tsave, exp_ops, options = options)

    # Extracting the results
    result0 = expect(exp_ops[0], result.states)
    result1 = expect(exp_ops[1], result.states)
    result2 = expect(exp_ops[2], result.states)
    return np.real(result0), np.real(result1), np.real(result2), I(tlist), Q(tlist)

def Gaussian(tlist, Kerr, tg):
    dim = 3
    a = destroy(dim)
    Kerr = Kerr *2*np.pi
    H0 = 0.5 * Kerr * a.dag() * a.dag() * a * a
    tlist = np.linspace(0, 100, 100)  # Example value, you can change it as needed

    H0 = -0.5 * Kerr * a.dag() * a.dag() * a * a
    H1s = [a + a.dag(), 1j * (a - a.dag())]

    time = tg
    tsave = tlist
    initial_states = [basis(dim, 0)]
    exp_ops = [basis(dim, idx) * basis(dim, idx).dag() for idx in range(dim)]

    def gaussian_env(t, sigma, matrix_element, tg, rot_angle=np.pi, ti=0):
        tf = ti + tg
        amp = rot_angle / (np.sqrt(2 * np.pi) * sigma * jsp.erf(tg / (2 * np.sqrt(2) * sigma)) - np.exp(-(tg / 2) ** 2 / (2 * sigma ** 2)) * tg) / matrix_element
        offset = amp * np.exp(-(tg / 2) ** 2 / (2 * sigma ** 2))
        return (amp * np.exp(-(t - ti - tg / 2) ** 2 / (2 * sigma ** 2)) - offset) * np.heaviside(t - ti, 0) * np.heaviside(tf - t, 0)
    def I(t):
        tg = time
        sigma = tg / 3
        matrix_element = 1
        rot_angle = np.pi
        return gaussian_env(t, sigma, matrix_element, tg, rot_angle) / 2

    def H_func(t, args):
        H = H0
        H += I(t) * H1s[0]
        return H

    H = QobjEvo(H_func, args={})
    options = {"store_states": True, "atol": 1e-12}
    result = sesolve(H, initial_states[0], tsave, exp_ops, options = options)
    # Extracting the results
    result0 = expect(exp_ops[0], result.states)
    result1 = expect(exp_ops[1], result.states)
    result2 = expect(exp_ops[2], result.states)
    return np.real(result0), np.real(result1), np.real(result2)



In [2]:
def DRAG_plot(ax, tg, anh):
    tlist = np.linspace(0, 50, 100)
    result, result1, result2, I, Q = DRAG(tlist, anh, tg)
    ax.plot(tlist, result, label='State |0⟩')
    ax.plot(tlist, result1, label='State |1⟩')
    ax.plot(tlist, result2, label='State |2⟩')
    ax.legend()
    ax.set_xlabel('Time')
    ax.set_ylabel('Expectation Value')
    ax.set_title('DRAG Expectation Values')

def pulse_plot(ax, tg, anh):
    tlist = np.linspace(0, 50, 100)
    result, result1, result2, I, Q = DRAG(tlist, anh, tg)
    ax.plot(tlist, I / (2 * np.pi), label='I Component')
    ax.plot(tlist, Q / (2 * np.pi), label='Q Component')
    ax.legend()
    ax.set_xlabel('Time')
    ax.set_ylabel('Amplitude')

def Gaussian_plot(ax, tg, anh):
    tlist = np.linspace(0, 50, 100)
    result, result1, result2 = Gaussian(tlist, anh, tg)
    ax.plot(tlist, result, label='State |0⟩')
    ax.plot(tlist, result1, label='State |1⟩')
    ax.plot(tlist, result2, label='State |2⟩')
    ax.legend()
    ax.set_xlabel('Time')
    ax.set_ylabel('Expectation Value')
    ax.set_title('Gaussian Pulse Expectation Values')

# Interactive plotting using `sr.InteractivePlot`
interactive_plot = sr.InteractivePlot(
    [[DRAG_plot, Gaussian_plot], [pulse_plot, None]],  # Grid of subplots
    [
        ["tg", 5, 50.0, 1, 13],  # Time array
        ["anh", 0.05, 0.5, 0.01, 0.14],  # Anharmonicity parameter
    ]
)


/Users/yunwei/Desktop/PycharmProjects/qutip/qutip/solver/solver_base.py:576: FutureWarning: e_ops will be keyword only from qutip 5.3 for all solver
  warnings.warn(


Dialog(children=[Card(children=[CardTitle(children=['Adjust Figure Settings'], layout=None), CardText(children…

Container(children=[Container(children=[Container(children=[Output(layout=Layout(overflow='hidden'))], class_=…